<a href="https://colab.research.google.com/github/aarongraybill/Math210/blob/main/ProblemSets/Midterm1/Midterm1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math 210
## Aaron Graybill
### Midterm 1
### 3/18/21

## Problem 1.
First let me bring in all of the predefined scripts and packages:

In [2]:
import numpy as np

In [3]:
def print_tableau(a,indep_names,dep_names):
#
# Given matrix "a" and lists of variables names "indep_names" and "dep_names",
# this function prints the matrix and labels in standard tableau format
# (including adding the -1, the minus signs in the last column, and labeling the lower-right as obj)
#
# First, check the inputs: indep_names should be one shorter than the number of columns of A
#                          dep_names should be one shorter than the number of rows of A
#
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
    nindep = len(indep_names)
    ndep = len(dep_names)
    if nindep != ncols-1:
        print("WARNING: # of indep vbles should be one fewer than # columns of matrix")
    if ndep != nrows-1:
        print("WARNING: # of dep vbles should be one fewer than # rows of matrix")
# Now do the printing (uses a variety of formatting techniques in Python)        
    for j in range(ncols-1):                    # Print the independent variables in the first row
        print(indep_names[j].rjust(10),end="")  # rjust(10) makes fields 10 wide and right-justifies;
                                                #    the end command prevents newline)
    print("        -1")                         # Tack on the -1 at the end of the first row
    for i in range(nrows-1):
        for j in range(ncols):                  # Print all but the last row of the matrix
            print("%10.3f" % a[i][j],end="") # The syntax prints in a field 10 wide, showing 3 decimal points
        lab = "= -" + dep_names[i]
        print(lab.rjust(10))
    for j in range(ncols):
        print("%10.3f" % a[nrows-1][j],end="")  # Print the last row of the matrix, with label "obj" at end
    lab = "= obj"
    print(lab.rjust(10))
    print(" ")    # Put blank line at bottom

In [4]:
def pivot(a,pivrow,pivcol,indep_names,dep_names) :
# 
# Given matrix "a", a row number "pivrow" and column number "pivcol", 
#  and lists of variable names "indep_names" and "dep_names", this
#  function does three things:
#    (1) outputs the new version of the matrix after a pivot,
#    (2) updates the lists of variable names post-pivot
#    (3) prints the new matrix, including labels showing the variable names
#
# First, check the inputs: indep_names should be one shorter than the number of columns of A
#                          dep_names should be one shorter than the number of rows of A
#                          you should not be pivoting on the last row or last column
#
    a = a.astype(float)   # make sure entries are treated as floating point numbers
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
    nindep = len(indep_names)
    ndep = len(dep_names)
    if nindep != ncols-1:
        print("WARNING: # of indep vbles should be one fewer than # columns of matrix")
    if ndep != nrows-1:
        print("WARNING: # of dep vbles should be one fewer than # rows of matrix")
    if pivrow > nrows-1 or pivcol > ncols-1:
        print("WARNING: should not pivot on last row or column")
    newa = a.copy()       # make a copy of A, to be filled in below with result of pivot
    p = a[pivrow-1][pivcol-1]   # identify pivot element
    newa[pivrow-1][pivcol-1] = 1/p   # set new value of pivot element
    # Set entries in p's row
    for j in range(ncols):
        if j != pivcol-1:
            newa[pivrow-1][j]=a[pivrow-1][j]/p;
    # Set entries in p's column
    for i in range(nrows):
        if i != pivrow-1:
            newa[i][pivcol-1]=-a[i][pivcol-1]/p;
    # Set all other entries
    for i in range(nrows):
        for j in range(ncols):
            if i != pivrow-1 and j != pivcol-1:
                r = a[i][pivcol-1]
                q = a[pivrow-1][j]
                s = a[i][j]
                newa[i][j]=(p*s-q*r)/p
    # Now swap the variable names
    temp = indep_names[pivcol-1]
    indep_names[pivcol-1]=dep_names[pivrow-1]
    dep_names[pivrow-1]=temp
    print_tableau(newa,indep_names,dep_names) # Print the matrix with updated labels
    return newa;

In [5]:
def target(a) :
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in "a"
    ncols = a.shape[1]
    import numpy as np
    v = np.empty(ncols-1)
    for i in range(ncols-1):
        v[i]=a[nrows-1,i]
    biggest_c = np.max(v)
    where_is_biggest_c = np.argmax(v)+1
    if biggest_c > 0 :
        return where_is_biggest_c
    else :
        return -1

In [36]:
def select(a,pivcolnum) :
    nrows = a.shape[0]    # use the shape function to determine number of rows and cols in A
    ncols = a.shape[1]
# First task: work down the column and record the b/a ratios in a vector v
#     except record -1 if a is negative or zero
    import numpy as np
    v = np.zeros(nrows-1)
    for i in range(nrows-1):
        if a[i,pivcolnum-1]>0 :
            v[i] = a[i,ncols-1]/a[i,pivcolnum-1]
        else :
            v[i] = -1
# Second task: if max b/a > -1, find min b/a by hand (ignoring zero entries in v)
    if np.max(v) > -1 :  
        min_so_far = np.max(v)+1  # Initialize variable to be for-sure bigger than the min
        for i in range(nrows-1):
            if v[i] > -1 and v[i] < min_so_far :
                min_so_far = v[i]
                where_is_min = i+1   # Add 1 to use human numbering
        return where_is_min      # Once we've scanned v for min, we can return result
    else :          
        return -1

In [7]:
# Create Simplex BF
def SimplexBF(a,indep_names,dep_names):
  nrows, ncols = a.shape
  a_new = a
  print_tableau(a_new,indep_names,dep_names)
  while np.max(a_new[nrows-1,:-1])>0:
    pivcol=target(a_new)
    pivrow=select(a_new,pivcol)
    if pivrow == -1:
      return("Unbounded")
    else:
      a_new=pivot(a_new,pivrow,pivcol,indep_names,dep_names)
      print_tableau(a_new,indep_names,dep_names)



### a.
Okay now I can start on SimplexNBF:

In [8]:
def targetnbf(tab):
  nrows, ncols = tab.shape
  new_i = -1
  for i in range(nrows-1) :# don't check obj fn row
    if tab[i,ncols-1] < 0:
      new_i = i+1
  return(new_i)

### b.

In [9]:
def candidateone(tab,targetedrow):
  #don't specify a row that's the obj fn row, I have no protocol against it
  nrows, ncols = tab.shape
  for i in range(ncols-1): #don't check last col bc it's the b's
    if tab[targetedrow-1,i]<0 :
      return(i+1)#bailout if found one
  return(-1) #none found in loop, return -1


### c.

In [10]:
def  selectnbf(tab,targetedrow,pivcolumn) :
  nrows, ncols = tab.shape

  #subset of two columns in question exclude obj fn row
  candidate_column =tab[targetedrow-1:nrows-1,pivcolumn-1]
  b_column=tab[targetedrow-1:nrows-1,ncols-1]

  # compute ratios
  ratios=b_column/candidate_column

  # Find row in subset and convert back to index in full table:
  #don't need to start at first row, let those be the starting values
  cur_ratio= ratios[0]
  best_row=0
  # the seemingly misplaced -1s and +1s are because we start at second row
  for i in range(len(candidate_column)-1):
    if candidate_column[i+1]>0 and ratios[i+1]<cur_ratio :
      cur_ratio=ratios[i+1]
      best_row=i+1

  #the targetedrow already has the +1, so this converts to start at 1
    #and converts table subset index to full tableau index
  best_row=best_row+targetedrow  

  return([best_row,pivcolumn]) #output

### d.

In [21]:
def simplexnbf(tab,indep_names,dep_names):
  nrows, ncols = tab.shape #get dims
  current_target_row=np.Inf #set current target row to nonsense
  tab_new=tab # get primer value for tableau iterator
  
  NotReadyForBF = True #changed when problem is BF
  SolutionPossible = True #false when no solution is discovered

  print_tableau(tab,indep_names,dep_names) #print initial tableau

  # while NBF and solution still possible apply NBF rules
  while NotReadyForBF and SolutionPossible:
    current_target_row=targetnbf(tab_new) # find target
    if current_target_row ==-1 : #if target is -1, Ready for BF
      NotReadyForBF = False
    else : #find candidate from targeted row
      current_candidate_column=candidateone(tab_new,current_target_row)
      if current_candidate_column ==-1 : #if no candidates, no solution
        SolutionPossible = False
      else : #pivot based off of the computed selection
        pivot_row,pivot_col=selectnbf(tab_new,current_target_row,current_candidate_column)
        tab_new=pivot(tab_new,pivot_row,pivot_col,indep_names,dep_names)
  if not NotReadyForBF : #if we bailed because ready for BF apply SimplexBF
    SimplexBF(tab_new,indep_names,dep_names)
  if not SolutionPossible: # if we bailed bc constraint set emtpy, say so 
    return("-1, Constraint set empty, sorry ☹️")
  
  #we can't exit the while loop for anything but the reasons above, 
  # so don't need any else statements
  


That's actually not what the question is asking for, so let me tighten it up to give the requested output (though you can gather the desired result from the above).

In [12]:
def simplexnbf_final(tab,indep_names,dep_names):
  output = simplexnbf(tab,indep_names,dep_names)
  # my sad error message is a string, so when we see that, return -1
  if output=="-1, Constraint set empty, sorry ☹️": 
    return -1
  #else we must have a tableau for BF, return 0
  else : 
    return 0

Here is proof that all of my functions work as desired.

In [32]:
a=np.array([
            [-2,-10,3,-20,8],
            [-1,1,0,-3,-6],
            [1,4,-1,8,-1],
            [1,1,1,1,0]])
indep_names=["x","y","z","w"]
dep_names=["t1","t2","t3"]

print(targetnbf(a))
print(candidateone(a,3))
print(selectnbf(a,3,1))


simplexnbf(a,indep_names,dep_names)

3
3
[3, 1]
         x         y         z         w        -1
    -2.000   -10.000     3.000   -20.000     8.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
     1.000     4.000    -1.000     8.000    -1.000     = -t3
     1.000     1.000     1.000     1.000     0.000     = obj
 
         x         y        t3         w        -1
     1.000     2.000     3.000     4.000     5.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
    -1.000    -4.000    -1.000    -8.000     1.000      = -z
     2.000     5.000     1.000     9.000    -1.000     = obj
 
        t2         y        t3         w        -1
     1.000     3.000     3.000     1.000    -1.000     = -t1
    -1.000    -1.000    -0.000     3.000     6.000      = -x
    -1.000    -5.000    -1.000    -5.000     7.000      = -z
     2.000     7.000     1.000     3.000   -13.000     = obj
 


'-1, Constraint set empty, sorry ☹️'

In [34]:
indep_names=["x","y","z","w"]
dep_names=["t1","t2","t3"]

simplexnbf_final(a,indep_names,dep_names)

         x         y         z         w        -1
    -2.000   -10.000     3.000   -20.000     8.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
     1.000     4.000    -1.000     8.000    -1.000     = -t3
     1.000     1.000     1.000     1.000     0.000     = obj
 
         x         y        t3         w        -1
     1.000     2.000     3.000     4.000     5.000     = -t1
    -1.000     1.000     0.000    -3.000    -6.000     = -t2
    -1.000    -4.000    -1.000    -8.000     1.000      = -z
     2.000     5.000     1.000     9.000    -1.000     = obj
 
        t2         y        t3         w        -1
     1.000     3.000     3.000     1.000    -1.000     = -t1
    -1.000    -1.000    -0.000     3.000     6.000      = -x
    -1.000    -5.000    -1.000    -5.000     7.000      = -z
     2.000     7.000     1.000     3.000   -13.000     = obj
 


-1

## Problem 2

### a.
Since we pivot on $a_{62}$ using the pivot formula we have the following values for the new $b_i$s:

$$
b_i^{new}=
\begin{cases}
\frac{b_i}{a_{62}} & i =6\\
\frac{a_{62}b_i-a_{i2}b_6}{a_{62}} & i\neq6
\end{cases}
$$
Plugging in the zeros and simplyfing gives:

$$
b_i^{new}=
\begin{cases}
0 & i =6\\
b_i & i\neq6
\end{cases}
$$

But since $b_6=0$, we are back where we started with all of the $b$s reamining the same. 

### b.
Let $a_{ij}^n$ be the value $a_{ij}$ in tableau $T_n$.

Proof by contradiction, suppose that $a_{62}^2$ was indeed the pivot for $T^2$. Since all of the $b_i^2$s remain the same in $T_2$ as the $b_i^1$s in $T_1$. we must again target row four.

To pivot on $a_{62}^2$, by the candidate rules, $a_{42}^2<0$.

Since we pivoted on $a_{62}^1$, $a_{62}^1>0$ (bc $b_6>0$ or it would be target). For $T_1$, we targeted row 4, so $a_{42}^1<0$ again by the candidate rules.

Using the pivot rules, I compute $a_{42}^2$ as follows:
$$
a_{42}^2=-\frac{a_{42}^1}{a_{62}^1}
$$
From the results above the numerator must be negative while the denominator must be positive, so the whole fraction when multiplied by $-1$ is positive. Therefore $a_{42}^2>0$. Which contradicts the earlier result. Therefore the supposition that $a_{62}^2$ is the pivot must be incorrect. We pivot elsewhere.  $\blacksquare$

## Problem 3.

There are two ways to have no minimum. First if the constraint set is empty which in this case it is not (see plot below). Or if the objective function is unconstrained in a direction that decreases the objective function to decrease ad infiinitum. For the function objective function $y-Ax$ to be decreasing while $x,y\geq0$, at least one of those terms has to be negative. Therefore $A>0$. 

Now if the max exists, it must occur at a finite point. By the geometric method, we know that this point must be at the intersection of wto constraints (because there are two indep vars). The plot below outlines which intersections form the corners of the region.


In [47]:
%%HTML
<iframe src="https://www.desmos.com/calculator/6y1stgwg3x" width="1000px" height="500px" style="border: 1px solid #ccc" frameborder=0></iframe>

Reading off of the graph above, we have that the corners at which a maximum could occur are at the intersection of the edges of the 1st and 3rd constraints and the 2nd and 3rd constraints.

Solving these systems gives that the maximum must occcur at either $\left(\frac{28}{5},\frac{92}{5}\right)$ or $\left(\frac{69}{4},\frac{27}{4}\right)$

I will now show the extant maximum unbounded min property occurs whenever $A>4$. I spent a long, fruitless, time trying to come up with a concise algebraic proof, but alas I will have to appeal to graphical inuition.

First suppose that $A\leq4$, I will show that an arbitrarily high value $C$ can be attained. When $A\leq4$ as plotted below, note that the slope of the contour is less than that of the upper part of the constraint set. Note that along this contour $C=y-Ax\implies y=Ax+C$, so the contour value is only affect the $y$-intercept of the contour, increasing the $y$-intercept might push the intersection to a different place, but since the slope of the contour is less than that of the top of the constraint set, there must be an intersection, we can attain the desired $C$.

In [70]:
%%html
<iframe src="https://www.desmos.com/calculator/v6bdqqvagy" width="1000px" height="500px" style="border: 1px solid #ccc" frameborder=0></iframe>

This shows that $A\leq4$ cannot have the desired property, so we must see if we can find the desired property when $A>4$. First to show it has a maximum. Consider the plot below, whenever $A\geq4$, the contours move in the northwest direction as the value of the contour increases and the slope of this line is greater than the top part of the constraint, so eventually we will be left with just one point in the constraint set and by the geometric method, this must be the unique maximizer at $(5.6,18.4)$.

In [71]:
%%html
<iframe src="https://www.desmos.com/calculator/2916qk5amf?embed" width="1000px" height="500px" style="border: 1px solid #ccc" frameborder=0></iframe>

Now to show that the minimum is unbounded. The contours moving down go in the southeast direction, and since the slope of the contour is steeper than that of the bottom of the constraint set there always must be an intersection, we can find arbitrarily small values of the objective function. This completes the skeleton of the proof.

## Problem 4.
Thankfully, the problem is already in the proper form to be added to the tableau, though we will have to run NBF.
Note that the objective function is asking us to find the largest value of $x_4$ which means all of the other $x_i$s add nothing and each $x-4$ is worth one. The objective function must then be $f(X)=x_4$
I setup the tableau by reading off of the constraints and applying SimplexNBF: 

In [51]:
a=np.array([[1,2,1,1,16],[-1,-1,2,3,5],[-1,2,-4,5,-14],[1,-4,-3,6,24],[0,0,0,1,0]])
indep_names=["x1","x2","x3","x4"]
dep_names=["t1","t2","t3","t4"]
simplexnbf(a,indep_names,dep_names)

        x1        x2        x3        x4        -1
     1.000     2.000     1.000     1.000    16.000     = -t1
    -1.000    -1.000     2.000     3.000     5.000     = -t2
    -1.000     2.000    -4.000     5.000   -14.000     = -t3
     1.000    -4.000    -3.000     6.000    24.000     = -t4
     0.000     0.000     0.000     1.000     0.000     = obj
 
        t3        x2        x3        x4        -1
     1.000     4.000    -3.000     6.000     2.000     = -t1
    -1.000    -3.000     6.000    -2.000    19.000     = -t2
    -1.000    -2.000     4.000    -5.000    14.000     = -x1
     1.000    -2.000    -7.000    11.000    10.000     = -t4
     0.000     0.000    -0.000     1.000    -0.000     = obj
 
        t3        x2        x3        x4        -1
     1.000     4.000    -3.000     6.000     2.000     = -t1
    -1.000    -3.000     6.000    -2.000    19.000     = -t2
    -1.000    -2.000     4.000    -5.000    14.000     = -x1
     1.000    -2.000    -7.000    11.000    10.000

I will briefly talk through those pivots because the question seems to be asking us to figure out the pivots by hand. First we have to run NBF because one of the $b_i$s is negative. As such we target row 3, which makes us take row one as the candidates and then we compute smallest ratio which happens to be in $a_{31}$. Pivoting there gives the next tableau in the sequence, which happens to be BF. 

We target the column with the highest value, that happens to be column 4. I compute the ratios that have positive denominators, and we select the smallest which happens to be $a_{14}$. Continue in a similar way until we get the following result:
\begin{align}
x_1^*&=9.767 \\
x_2^*&=0 \\
x_3^*&=3.933 \\
x_4^*&=2.3 \\
\end{align}
And the value of the objective function is 2.3, implying that the maximum value that $x_4$ can attain in the set is $2.3$ 

## Problem 5.
We have five choice variables, $\mathcal{A}_1$ and $\mathcal{A}_2$ the number of apartments built in periods one and two, $\mathcal{O}_1$ and $\mathcal{O}_2$ for the number of of offices built in period one and two and  $\mathcal{T}$ for the number of electricians sent to training in period one
The master electricians in period one is a static number and they can be allocated to either $\mathcal{A}$ or $\mathcal{O}$. Their constraint would be:
$$
\mathcal{A}_1+\mathcal{O}_1\leq 20\\
$$

In period two, the constraint is eased by the number of electricians sent to training. Therefore the $t=2$ master electrician constraint is 
$$
\mathcal{A}_2+\mathcal{O}_2\leq 20+\mathcal{T}\\
$$.

We have a similar constraint but going in the opposite direction for regular electricians. We send $\mathcal{T}$ away in the first period meaning:
$$
2\mathcal{A}_1+5\mathcal{O}_1 \leq120-\mathcal{T}
$$
Those electricians become masters in period two, so are also removed from that constraint giving the same constraint for $t=2$
$$
2\mathcal{A}_2+5\mathcal{O}_2 \leq120-\mathcal{T}
$$
We then have the carptenter and painter constraints which are more straightforward and give:
\begin{align*}
4\mathcal{A}_1 + 2\mathcal{O}_1 &\leq 100 \\
4\mathcal{A}_2 + 2\mathcal{O}_2 &\leq 100 \\
5\mathcal{A}_1 + 2\mathcal{O}_1 &\leq 90 \\
5\mathcal{A}_2 + 2\mathcal{O}_2 &\leq 90
\end{align*}

The final thing to mention is the objective function, each apartment in either period brings in 25k and an office brings in 30k in either period. Each trainee subtracts 5k from profit, so our objective function in thousands is:
$$
25\mathcal{A}_1+25\mathcal{A}_2+30\mathcal{O}_1+30\mathcal{O}_2-5\mathcal{T}
$$
Let's tableau-ify and solve:


In [58]:
a=np.array([[1,1,0,0,0,20],
            [0,0,1,1,-1,20],
            [2,5,0,0,1,120],
            [0,0,2,5,1,120],
            [4,2,0,0,0,100],
            [0,0,4,2,0,100],
            [5,2,0,0,0,90],
            [0,0,5,2,0,90],
            [25,30,25,30,-5,0]])
indep_names=["A1","O1","A2","O2","T"]
dep_names=["mElec1","mElec2","Elec1","Elec2","Carp1","Carp2","Paint1","Paint2"]

simplexnbf(a,indep_names,dep_names)

        A1        O1        A2        O2         T        -1
     1.000     1.000     0.000     0.000     0.000    20.000 = -mElec1
     0.000     0.000     1.000     1.000    -1.000    20.000 = -mElec2
     2.000     5.000     0.000     0.000     1.000   120.000  = -Elec1
     0.000     0.000     2.000     5.000     1.000   120.000  = -Elec2
     4.000     2.000     0.000     0.000     0.000   100.000  = -Carp1
     0.000     0.000     4.000     2.000     0.000   100.000  = -Carp2
     5.000     2.000     0.000     0.000     0.000    90.000 = -Paint1
     0.000     0.000     5.000     2.000     0.000    90.000 = -Paint2
    25.000    30.000    25.000    30.000    -5.000     0.000     = obj
 
        A1        O1        A2        O2         T        -1
     1.000     1.000     0.000     0.000     0.000    20.000 = -mElec1
     0.000     0.000     1.000     1.000    -1.000    20.000 = -mElec2
     2.000     5.000     0.000     0.000     1.000   120.000  = -Elec1
     0.000     0.000    

Let's interpret the above results. We have first that the company attains a maximal profit of $\$1364.583$k dollars or $\$1,364,583$ in profit. To attain that profit, the company sends $8.75$ electricians to training. They produce $20$ offices and no apartments in period one, but in period two, they produce $17.917$ offices and $10.833$ apartments. Note that we send just enough electricians to training such that neither master electricians or electrians are left not working in period two.  

## Problem 6. 

Define the following: $B$ number of protein bars used, $A$ number of apples used, $P$ number of poptarts, $T$ number of twinkies and $H$ for hot pockets.

The constraints are fairly straightforward:

\begin{align*}
8B+A+2.3P+2T+19H&\geq300\\
130B+95A+209P+260T+600H&\geq10000 \\
130B+95A+209P+260T+600H&\leq20000 \\
120B+0A+172P+350T+1050H&\leq 15000 \\
4B+0A+7P+8T+26H&\leq 500 \\
3B+3A+.7P+0T+2H&\geq 100
\end{align*}

To CanonicalMax Form:
\begin{align*}
-8B-A-2.3P-2T-19H&\leq-300\\
-130B-95A-209P-260T-600H&\leq-10000 \\
130B+95A+209P+260T+600H&\leq20000 \\
120B+0A+172P+350T+1050H&\leq 15000 \\
4B+0A+7P+8T+26H&\leq 500 \\
-3B-3A-.7P-0T-2H&\leq -100
\end{align*}

But we are trying to find the minimum of the function so, we must also multiply the objective function by $-1$ giving:
$$
-1B -.8A -.75P -.6T- 1.75H
$$

tableauing and simplexing nbf gives:


In [67]:
a=np.array([[-8,-1,-2.3,-2,-19,-300],
            [-130,-95,-209,-260,-600,-10000],
            [130,95,209,260,600,10000],
            [120,0,172,350,1050,15000],
            [4,0,7,8,26,500],
            [-3,-3,-.7,0,-2,-100],
            [-1,-.8,-.75,-.6,-1.75,0]
            ])
indep_names=["ProtBar","Apple","PopTart","Twinkie","HotPock"]
dep_names=["Protein","Calories_low","Calories_high","Sodium","Fat","Fiber"]
simplexnbf(a,indep_names,dep_names)

   ProtBar     Apple   PopTart   Twinkie   HotPock        -1
    -8.000    -1.000    -2.300    -2.000   -19.000  -300.000= -Protein
  -130.000   -95.000  -209.000  -260.000  -600.000-10000.000= -Calories_low
   130.000    95.000   209.000   260.000   600.000 10000.000= -Calories_high
   120.000     0.000   172.000   350.000  1050.000 15000.000 = -Sodium
     4.000     0.000     7.000     8.000    26.000   500.000    = -Fat
    -3.000    -3.000    -0.700     0.000    -2.000  -100.000  = -Fiber
    -1.000    -0.800    -0.750    -0.600    -1.750     0.000     = obj
 
     Fiber     Apple   PopTart   Twinkie   HotPock        -1
    -2.667     7.000    -0.433    -2.000   -13.667   -33.333= -Protein
   -43.333    35.000  -178.667  -260.000  -513.333 -5666.667= -Calories_low
    43.333   -35.000   178.667   260.000   513.333  5666.667= -Calories_high
    40.000  -120.000   144.000   350.000   970.000 11000.000 = -Sodium
     1.333    -4.000     6.067     8.000    23.333   366.667    = -Fat
  

The least expensive way to feed the camper is using $19.369$ Apples, $12.360$ hot pockets, $5.725$ protein bars and using none of the other items.